# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - można zaimplementować w tym celu wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

## Działająca metoda Minimax

In [162]:
import math
import numpy as np
import alfa_beta as ai
from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove, ConnectFourState

def alphabeta(position, players, alpha, beta, max_player):
    if players == 0 or position == 'terminal':
        return evaluate_position(position)
    if current_player is max_player:
        max_eval = -math.inf
        for child in children(position):
            eval = alphabeta(child, players - 1, alpha, beta, False)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = math.inf
        for child in children(position):
            eval = alphabeta(child, players - 1, alpha, beta, True)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval


## metoda Minimax z obcinaniem alpha-beta

In [163]:
def evaluate_position(game_state):
    return game_state.get_heuristic()


def get_winner(game_state):
    return game_state.get_winner()


def current_player(game_state):
    return game_state.get_current_player()


def is_finished(game_state):
    return game_state.is_finished()


def alphabeta(game_state, players, alpha, beta, maximising_player):
    if players == 0 or is_finished(game_state):
        if is_finished(game_state):
            winner = get_winner(game_state)
            if winner is current_player(game_state):
                return 10e6, None
            elif winner is not (current_player(game_state) or None):
                return -10e6, None
            else:
                return 0, None
        else:
            return evaluate_position(game_state), None
    if maximising_player:
        max_eval = -math.inf
        possible_moves = game_state.get_moves()
        rng = np.random.default_rng()
        rng.shuffle(possible_moves)
        best_move = rng.choice(possible_moves)
        for move in possible_moves:
            state = game_state.make_move(move)
            eval = alphabeta(state, players - 1, alpha, beta, False)[0]
            if eval > max_eval:
                max_eval = eval
                best_move = move
            elif max_eval == eval:
                rng = np.random.default_rng()
                best_move = rng.choice([best_move, move])
            alpha = max(alpha, max_eval)
            if alpha >= beta:
                break

        return max_eval, best_move
    else:
        min_eval = math.inf
        possible_moves = game_state.get_moves()
        rng = np.random.default_rng()
        rng.shuffle(possible_moves)
        best_move = rng.choice(possible_moves)
        for move in possible_moves:
            state = game_state.make_move(move)
            eval = alphabeta(state, players - 1, alpha, beta, True)[0]
            if eval < min_eval:
                min_eval = eval
                best_move = move
            elif min_eval == eval:
                rng = np.random.default_rng()
                best_move = rng.choice([best_move, move])
            min_eval = min(min_eval, eval)
            beta = min(beta, min_eval)
            if alpha >= beta:
                break
        return min_eval, best_move


In [164]:
ROW_COUNT = 6
COLUMN_COUNT = 7

## Uruchomienie gry

In [165]:
def GameStart(player_1, player_2, GameWithPrint, interactive):
    player1 = Player('a')
    player2 = Player('b')
    game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT),first_player=player1, second_player=player2)
    game.make_move(ConnectFourMove(3))
    game.make_move(ConnectFourMove(4))
    game.make_move(ConnectFourMove(3))

    if GameWithPrint:
        print(str(game))
    while not game.is_finished():
        game_state = game.state
        if interactive:
            a_points, a_move = ai.alphabeta(game_state, player_1, -math.inf, math.inf, True)
            game.make_move(a_move)
            print(f"Wartość ruchu a: {a_points} \n")
            print(str(game))

        else:
            a_points, a_move = ai.alphabeta(game_state, player_1, -math.inf, math.inf, True)
            b_points, b_move = ai.alphabeta(game_state, player_2, -math.inf, math.inf, False)
            if GameWithPrint:
                game.make_move(a_move)
                print(f"Wartość ruchu a: {a_points} \n")
                print(str(game))
                game.make_move(b_move)
                print(f"Wartość ruchu b: {b_points} \n")
                print(str(game))
            else:
                game.make_move(a_move)
                game.make_move(b_move)

    winner = game.get_winner()
    if winner == player1:
        if GameWithPrint:
            print("\nWinner a")
        return 'player1'
    elif winner == player2:
        if GameWithPrint:
            print("\nWinner b")
        return 'player2'
    else:
        if GameWithPrint:
            print("\nDraw-game")
        return 'r'

In [166]:
GameStart(1, 2, True, False)

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][a][b][ ][ ]
Wartość ruchu a: 2 

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][b][ ][ ]
[ ][ ][ ][a][b][ ][ ]
Wartość ruchu b: 0 

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][b][ ][ ]
[a][ ][ ][a][b][ ][ ]
Wartość ruchu a: 4 

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][ ][a][b][ ][ ]
[a][ ][ ][a][b][ ][ ]
Wartość ruchu b: 2 

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][ ][a][b][ ][ ]
[a][ ][a][a][b][ ][ ]
Wartość ruchu a: 9 

Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[b][ ][ ][ ][ ][ ][ ]
[b][ ][ ][a][b][ ][ ]
[a][ ][a][a][b][ 

'player1'

##Statystyka

In [167]:
def statistics(player_1, player_2, GamePrint):
    GameStartNumber = 20 
    sum_a = sum_b = sum_r = 0
    
    for i in range(GameStartNumber):
        winner = GameStart(player_1, player_2, False, False)
        if winner == 'player1':
            sum_a += 1
        elif winner == 'player2':
            sum_b += 1
        else:
            sum_r += 1
   
    aPct = round(100 * sum_a / GameStartNumber)
    bPct = round(100 * sum_b / GameStartNumber)
    rPct = round(100 * sum_r / GameStartNumber)

    if GamePrint:
        print(f"Wynik gry = wygral a {aPct}%; wygral b {bPct}%; remisy {rPct}%.")
    return aPct, bPct, rPct

In [168]:
statistics(1, 1, False)

(100, 0, 0)

In [169]:
Output = np.empty([2, 2])

for i in range(2):
    for j in range (2):
        print(f"player1, player2 = {j+1, i+1}")
        res = statistics(j+1, i+1, True)
        Output [j, i] = res[0]
    print("\n")

player1, player2 = (1, 1)
Wynik gry = wygral a 100%; wygral b 0%; remisy 0%.
player1, player2 = (2, 1)
Wynik gry = wygral a 70%; wygral b 25%; remisy 5%.


player1, player2 = (1, 2)
Wynik gry = wygral a 95%; wygral b 5%; remisy 0%.
player1, player2 = (2, 2)
Wynik gry = wygral a 25%; wygral b 55%; remisy 20%.


